In [1]:
import torch
from transformers import pipeline

In [2]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda:0


In [3]:
## Execute only if 'bert-base-multi' is not in working directory
# qa_pipeline = pipeline(
#         "question-answering",
#         model="henryk/bert-base-multilingual-cased-finetuned-polish-squad2",
#         tokenizer="henryk/bert-base-multilingual-cased-finetuned-polish-squad2"
#     )

## Saving model to file

In [4]:
## Execute only if 'bert-base-multi' is not in working directory
# torch.save(qa_pipeline, 'bert-base-multi')

In [5]:
qa_pipeline = torch.load('bert-base-multi')

In [33]:
qa_pipeline({
        'context': "Warszawa jest największym miastem w Polsce pod względem liczby ludności i powierzchni",
        'question': "Jakie jest największe miasto w Polsce?"})

{'score': 0.998785138130188, 'start': 0, 'end': 8, 'answer': 'Warszawa'}

In [7]:
def getAnswer(context, query, only_ans=True):
    result = qa_pipeline({
        'context': context,
        'question': query
    })
    if only_ans:
        return result['answer']
    else:
        return result

In [34]:
with open('sklep.txt', 'r', encoding="utf-8") as file:
    data = file.read().replace('\n', ' ')
print(data)

Witamy w naszym sklepie Trendylook XD. W naszej ofercie posiadamy wiele markowych butów i ubrań. Oferujemy najlepsze ceny i darmowe zwroty do 14-dni. Białe buty adidas. Cena 249 PLN. Dostawa gratis od 200 PLN. Czerwone botki skórzane. Cena 339 PLN. Dostawa gratis od 200 PLN. Czarne buty nike. Cena 249 PLN. Dostawa gratis od 200 PLN. Czerwone buty Reebook. Cena 249 PLN. Dostawa gratis od 200 PLN. Buty kozaki eleganckie skórzane. Cena 249 PLN. Dostawa gratis od 200 PLN.  


In [41]:
getAnswer("ojciec Janka ma na imię Kuba, a tata Kuby nazywa się Andrzej.", "Jak nazywa się tata taty Janka?", onlyAns=False)

{'score': 0.46872636675834656, 'start': 53, 'end': 60, 'answer': 'Andrzej'}

In [35]:
getAnswer(context=data, query="czerwone buty?", only_ans=False)

{'score': 0.00603965949267149, 'start': 349, 'end': 356, 'answer': 'Reebook'}

In [40]:
getAnswer(context=data, query="szukam czerwone botki", only_ans=False)

{'score': 3.833120354101993e-05,
 'start': 210,
 'end': 233,
 'answer': 'Czerwone botki skórzane'}

In [42]:
getAnswer(context=data, query="czerwone botki cena", only_ans=False)

{'score': 0.008379863575100899, 'start': 240, 'end': 247, 'answer': '339 PLN'}

In [21]:
import openai

# load and set our key
openai.api_key = open("../key.txt", "r").read().strip("\n")

In [29]:
class ChatWithGPT:
    def __init__(self):
        self.message_log = []

    def format_query(self, question):
        return str(question.replace('"', ''), encoding='UTF-8')

    def ask_chat_gpt(self, question):
        try:
            question = self.format_query(question)
        except:
            return "Please enter valid question"
        self.message_log.append({"role": "user", "content": f"{question}"})
        try:
            completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", # this is "ChatGPT" $0.002 per 1k tokens
            messages=self.message_log
            )
        except:
            return "Unfortunately Chatgpt is unavailable right now :("

        assistant_reply = completion.choices[0].message.content
        self.message_log.append({"role": "assistant", "content": f"{assistant_reply.encode(encoding='UTF-8')}"})

        return assistant_reply

In [33]:
test = ChatWithGPT()
query = input()
ans1 = test.ask_chat_gpt(query)

In [34]:
print(test.message_log)

[{'role': 'user', 'content': "podaj najpopularniejsze ' '  auto w polsce"}, {'role': 'assistant', 'content': '\n\nNajpopularniejsze auto w Polsce to Volkswagen Golf.'}]


In [23]:
session = ChatWithGPT()
query = "odpowiedz na pytanie 'Jaki jest status mojego zamówienia nr 345245?' pełnym zdaniem, gdzie odpowiedzią jest 'wysłana'."
ans = session.ask_chat_gpt(query)


# "odpowiedz na pytanie 'Jaki jest status mojego zamówienia nr 345245?' pełnym zdaniem, gdzie odpowiedzią jest 'wysłana'."
# "Answer for a question in polish: 'Jaki jest status mojego zamówienia nr 345245?' in full sentence, where answer is: 'wysłana'."

In [24]:
print(ans)



Twoje zamówienie nr 345245 zostało już wysłane.


In [7]:
print(completion)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Twoje zam\u00f3wienie nr 345245 zosta\u0142o ju\u017c wys\u0142ane.",
        "role": "assistant"
      }
    }
  ],
  "created": 1678997233,
  "id": "chatcmpl-6uoATVmsq6Qc7CHaEc54h0ClrzFrA",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 47,
    "total_tokens": 64
  }
}
